In [45]:
import re
import json
import logging
from pathlib import Path
from datetime import datetime
from datetime import date

# Custom JSON encoder to handle date serialization
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, date):
            return obj.isoformat()  # Convert date to ISO 8601 string
        return super().default(obj)


# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Italian month mapping for date parsing
MONTHS_IT = {
    "gennaio": 1, "febbraio": 2, "marzo": 3, "aprile": 4, "maggio": 5, "giugno": 6,
    "luglio": 7, "agosto": 8, "settembre": 9, "ottobre": 10, "novembre": 11, "dicembre": 12
}

# Helper function to parse Italian dates
def parse_italian_date(date_str):
    match = re.search(r"(\d{1,2})\s+(\w+)\s+(\d{4})", date_str, re.IGNORECASE)
    if match:
        day, month_str, year = match.groups()
        month = MONTHS_IT.get(month_str.lower())
        if month:
            return datetime(int(year), month, int(day)).date()
    return None

def extract_details(content):
    details = {
        "Document Type": None,
        "Couple": {
            "Spouse 1": None,
            "Spouse 2": None
        },
        "Marriage Details": {
            "Date": None,
            "Place": None,
            "Witnesses": []
        },
        "Parent Names": {
            "Spouse 1 Parents": None,
            "Spouse 2 Parents": None
        },
        "Annotations": [],
        "Additional Notes": None
    }

    try:
        # Extract document type
        doc_type_match = re.search(r"\*\*CERTIDÃO DE CASAMENTO\*\*", content, re.IGNORECASE)
        if doc_type_match:
            details["Document Type"] = "Marriage Certificate"

        # Extract couple's names
        couple_match = re.search(r"casamento de ([\w\s]+) e ([\w\s]+)\.", content, re.IGNORECASE)
        if couple_match:
            details["Couple"]["Spouse 1"] = couple_match.group(1).strip()
            details["Couple"]["Spouse 2"] = couple_match.group(2).strip()

        # Extract parent names
        spouse1_parents_match = re.search(r"filho de ([\w\s]+) e ([\w\s]+)", content, re.IGNORECASE)
        if spouse1_parents_match:
            details["Parent Names"]["Spouse 1 Parents"] = {
                "Father": spouse1_parents_match.group(1).strip(),
                "Mother": spouse1_parents_match.group(2).strip()
            }

        spouse2_parents_match = re.search(r"filha de ([\w\s]+) e ([\w\s]+)", content, re.IGNORECASE)
        if spouse2_parents_match:
            details["Parent Names"]["Spouse 2 Parents"] = {
                "Father": spouse2_parents_match.group(1).strip(),
                "Mother": spouse2_parents_match.group(2).strip()
            }

        # Extract marriage date and place
        marriage_date_match = re.search(r"casamento foi realizado aos ([\d\s\w]+),", content, re.IGNORECASE)
        if marriage_date_match:
            details["Marriage Details"]["Date"] = parse_italian_date(marriage_date_match.group(1).strip())

        marriage_place_match = re.search(r"na Paróquia de ([\w\s]+)", content, re.IGNORECASE)
        if marriage_place_match:
            details["Marriage Details"]["Place"] = marriage_place_match.group(1).strip()

        # Extract witnesses
        witnesses_match = re.findall(r"testemunhas ([\w\s]+) e ([\w\s]+)", content, re.IGNORECASE)
        if witnesses_match:
            for witness in witnesses_match[0]:
                details["Marriage Details"]["Witnesses"].append(witness.strip())

        # Extract annotations
        annotations_match = re.findall(r"- Aos ([\d/]+), ([\w\s]+);", content, re.IGNORECASE)
        for date, note in annotations_match:
            details["Annotations"].append({
                "Date": parse_italian_date(date.replace("/", " ")),
                "Note": note.strip()
            })

        # Additional notes
        additional_notes_match = re.search(r"Eu, abaixo assinado, extraí o presente documento[\s\S]+", content, re.IGNORECASE)
        if additional_notes_match:
            details["Additional Notes"] = additional_notes_match.group(0).strip()

    except Exception as e:
        logging.error(f"Error extracting details: {e}")

    return details

# Main function to process text
def process_text(content):
    return extract_details(content)

def save_to_json(data, output_path):
    try:
        with open(output_path, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4, cls=CustomJSONEncoder)
        logging.info(f"Data successfully saved to {output_path}")
    except Exception as e:
        logging.error(f"Failed to save data to {output_path}: {e}")

    parsed_data = process_text(input_text)
    print(json.dumps(parsed_data, ensure_ascii=False, indent=4, cls=CustomJSONEncoder))

if __name__ == "__main__":
    # Example input
    input_text = """
Ilmo(a). Sr(a) Oficial de Registro Civil das Pessoas Naturais da 11ª Circunscrição do  
Registro Civil das Pessoas Naturais e Tabelionato da Capital - RJ.  

Eu, **Rita de Cássia Gonçalves Freitas**, brasileira, estudante, residente à Rua Bento Lisboa, 89, Ap. 907,  
Catete, Rio de Janeiro - RJ, telefone (21) XXXX-XXXX, nascida em 21/02/1954,  
venho, respeitosamente, requerer a V.Sa., a instauração de procedimento para  
alteração de nome, nos termos do artigo 57, inciso II ou inciso III, da Lei nº 6.015/1973.  

**Declaro sob as penas da lei:**  
1. Que a alteração requerida está de acordo com meu nome atual, ao qual respondo civil e criminalmente.  
2. Que não sou parte em ação judicial sobre alteração de sobrenome ou, caso exista, a mesma já foi devidamente arquivada, conforme certidão em anexo.  

**Fundamento Jurídico:**  
O presente requerimento está fundamentado no artigo 57, inciso II ou inciso III, da Lei nº 6.015/1973.  

**Alteração de Nome:**  
Com fundamento nos termos legais, solicito a inclusão do sobrenome **David**, de forma que meu novo nome completo seja:  
**Rita de Cássia Freitas David.**  

**Registro:**  
- Livro: Livro E  
- Página: Fls. 226  

Certifico e dou fé que a assinatura supra foi lançada em minha presença.  

Rio de Janeiro, RJ  
**Assinatura do Requerente:** ______________________   
    """

    # Process the input text
    parsed_data = process_text(input_text)

    # Define the output path
    output_path = Path("/home/luisvinatea/Dev/Repos/genealogy/reports/declaraçao_sobrenome_casada_rita_de_cassia_freitas_david.json")

    # Save the parsed data to the output path
    save_to_json(parsed_data, output_path)



2025-01-24 22:22:39,918 - INFO - Data successfully saved to /home/luisvinatea/Dev/Repos/genealogy/reports/declaraçao_sobrenome_casada_rita_de_cassia_freitas_david.json


{
    "Document Type": null,
    "Couple": {
        "Spouse 1": null,
        "Spouse 2": null
    },
    "Marriage Details": {
        "Date": null,
        "Place": null,
        "Witnesses": []
    },
    "Parent Names": {
        "Spouse 1 Parents": null,
        "Spouse 2 Parents": null
    },
    "Annotations": [],
    "Additional Notes": null
}
